<a href="https://colab.research.google.com/github/andrew-veriga/Tensorflow-labs/blob/master/PrepareFeaturesLabels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Setup

подключаем знакомые нам `tensorflow`, `numpy` и библиотеку рисования графиков 

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

Мы используем здесь класс `tf.data.Dataset`, и чтобы создать себе некоторые данные, сделаем диапазон из 10 последовательных чисел. Будет удобно контролировать правильность наших действий, учитывая, что эти ряды должны равномерно возрастать. В реальных рядах такого удобства не будет, поэтому лучше заранее всё отладить на понятных числах.

In [ ]:
dataset = tf.data.Dataset.range(10)
for val in dataset:
   print(val.numpy())


Итак, теперь сделаем это немного интереснее. Мы будем использовать `dataset.window`, чтобы расширить наш набор данных с помощью сдвигающегося окна. Первый параметр `size` указывает размер окна, параметр `stride` указывает шаг, с которым отбираются значения, а `shift` - величину сдвига окна. Чтобы хорошо понять, как всё это работает, поэкспериментируйте с этими параметрами функции `window` поменяйте их в разных сочетаниях и посмотрите, что получится.\
*один фрагмент, выделенный окном в этом примере, выводится как горизонтальный ряд чисел, весь полученный набор должен выглядеть как стопка горизонтальных массивов*

In [ ]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, stride=1,shift=1)
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(), end=" ")
  print()

Мы получили выходные данные первых пяти элементов, а затем вторых пяти элементов, а затем третьих пяти элементов и т. д. В конце набора данных, когда данных недостаточно, чтобы дать нам пять элементов, вы увидите более короткие строки. Но это не то, что нужно, ведь все признак-векторы должны быть одинаковой длины.\
Чтобы отбросить фрагменты с недостаточным количеством значений, мы установим параметр `drop_reminder` в значение true.


In [ ]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(), end=" ")
  print()

Теперь давайте поместим эти данные в списки, чтобы можно было начать использовать их в машинном обучении.\
`TensorFlow` любит, чтобы его данные были в формате `numpy`.\
 Ну, мы можем легко преобразовать его, вызвав метод `.numpy()`, и когда распечатаем его, увидим, что он теперь указан в квадратных скобках.

In [ ]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda w: w.batch(5))
for w in dataset:
  print(w.numpy())


Хорошо, теперь нам надо разделить данные на признак-векторы и метки. В каждом элементе надо для признак-вектора оставить все значения, кроме последнего, которое будет меткой. И это может быть достигнуто с помощью функции `map()`

In [ ]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
# мы разбиваем каждый массив на две части: 
# первая содержит все элементы кроме последнего: [:-1], а вторая содержит только последний элемент: [-1:]
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
for x,y in dataset:
  print(x.numpy(), y.numpy())

и теперь наш набор данных выглядит как хороший набор признаков и меток.\
И осталось только перемешать его. Это поможет нам избежать предвзятости в  последовательности. \
И это возможно с помощью метода `.shuffle()`. Мы вызовем его с параметром `buffer_size=10`, потому что это то максимальное количество строк, которые у нас могут быть.\
Многократные прогоны покажут строки в разной последовательности, потому что они перемешиваются случайным образом.

In [ ]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda w: w.batch(5))
dataset = dataset.map(lambda w: (w[:-1], w[-1:]))
dataset = dataset.shuffle(buffer_size=10)
for x,y in dataset:
  print(x.numpy(), y.numpy())

*предвзятость (bias) последовательности - это когда порядок может влиять на выбор. Например, если бы я спросил вас, какой ваш любимый сериал и перечислил в таком порядке:«Игра престолов», «Убийство Евы», «Путешественники» и «Доктор Кто» , вы, скорее всего, выберете «Игра Престолов», так он вам знаком и он стоит первым. Даже если вам одинаково нравятся и другие сериалы. При обучении данных в наборе данных мы не хотим, чтобы последовательность влияла на обучение аналогичным образом, поэтому перемешать их бывает полезно.*

Наконец, пакетирование данных. и это делается с помощью метода `.batch()'.\
Устанавливая размер пакета в 2, мы объединим наши данные  по два x и два y вместе. Ранее мы не пользовались таким разбиением данных на пакеты, keras все делал за нас, если это было нужно. Но теперь попробуем другой подход

In [ ]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda w: w.batch(5))
dataset = dataset.map(lambda w: (w[:-1], w[-1:]))
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1)
for x,y in dataset:
  print("x = ", x.numpy())
  print("y = ", y.numpy())


Итак, это ноутбук с кодом, который разбивает временной ряд данных скользящим окном на размеченные признак-векторы. Попробуйте поиграть с параметрами в последней ячейке в тех методах, которые вам показались непонятными и как только вы хорошо разберетесь с тем, что тут происходит, переходите к следующей главе.
Там вы увидите сезонный набор данных, и узнаете, как с помощью техники скользящего окна получить из него нужные x и y для нейронной сети, чтобы обучить ее прогнозированию.